In [1]:
#Compare the reasoning here to the reasoning in the book. We could say:
#We divide the string into equally sized blocks of 3N digits, where 3N is the largest
#multiple of three such that the number 127127…127 with 3N digits does not exceed n.
#This is because we're working with more than 26 possible characters.
#We're doing the whole 128 ASCII charachters.

def calc_block_size(n):
  #First step is finding the number of digits of "n" the dead simple way. 
  #Notice that the log10(n) method for finding the number of digits is much faster, but falls victim to floating point precision for large n. 
  num_digits = len(str(n))

  #We now need to find the largest block size for which no matter the ASCII codes it contains, the block will never exceed n in value.
  #All ASCII codes will either be 3 digits long or written with leading zeros to be 3 digits long.
  #Thus the block size is some multiple of three.
  #First the case where the number of digits of n is already a multiple of three.
  if num_digits % 3 == 0:
  #Then if the maximum value condition for the block is broken, it will have to be the next lower multiple of three.
    if int("127"*(num_digits//3)) > n :
      block_size = num_digits - 3
  #Else the maximum value condition is never broken, and we can use the same number of digits as n. 
    else:
      block_size = num_digits
  
  #Second, if the number of digits of n is not a multiple of three to begin with, we can safely use the next lower multiple of three.
  else:
    block_size = 3 * (num_digits//3)

  return block_size

In [2]:
def process_line(line, block_size):
  #ord() gives us the "ordinal" (ASCII) code for each character. Thank you Bedir for teaching me that.
  #f"{:03d}" is an f-string (formatting). It's giving us the leading zeros.
  integer_sequence = "".join([f"{ord(x):03d}" for x in line])
  #We slice the sequence into blocks using [i:i+block_size]
  blocks = [integer_sequence[i:i+block_size] for i in range(0,len(integer_sequence),block_size)]
  #Finally we make the last block the same size as other blocks. Using NULL characters.
  blocks[-1] = blocks[-1] + (block_size - len(blocks[-1]) ) * "0"
  
  return blocks

  #This looks funky. But I assure you, this is perfectly Pythonic. 
  #Those are standard (highly performant) very readable Python list comprehensions.

In [3]:
p = 179424673
q = 236887691
n = p*q
print(n)
e = 275604541
d = 7833083281815061

42503496495400043


In [4]:
block_size = calc_block_size(n)
print(block_size)

15


In [5]:
def mod_exp(b, d, modulus):
  d = bin(d)[2:]
  r = 1  #remainder
  for i in range(len(d) - 1, -1, -1):
    r = (r * b ** int(d[i])) % modulus
    b = (b ** 2) % modulus
  return r

In [6]:
def encrypt_file():
  input_file_name = input("Enter file name: ")
  input_file = open(input_file_name, 'r')

  output_file_name = input_file_name + "Encrypted"
  output_file = open(output_file_name, 'w')

  for line in input_file:
    blocks = process_line(line, block_size)
    ciphered_line = " ".join([str(mod_exp(int(block), e, n)) for block in blocks])
    output_file.write(ciphered_line + '\n')
  
  input_file.close()
  output_file.close()

In [7]:
def decrypt_file():
  input_file_name = input("Enter file name: ")
  input_file = open(input_file_name, 'r')

  output_file_name = input_file_name + "Decrypted"
  output_file = open(output_file_name, 'w')

  for ciphered_line in input_file:
    deciphered_blocks = [str(mod_exp(int(c), d, n)) for c in ciphered_line.split(" ")]
    deciphered_blocks = [[int(block[i:i-3:-1][::-1]) for i in range(-1, -block_size, -3)][::-1] for block in deciphered_blocks]
    deciphered_blocks[-1] = [code for code in deciphered_blocks[-1] if code != 0]
    recovered_line = "".join([chr(code) for block in deciphered_blocks for code in block])

    output_file.write(recovered_line)
  
  input_file.close()
  output_file.close()

In [8]:
encrypt_file()

Enter file name:  MATH 308 Project Task One.txt


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 2177: invalid start byte

In [12]:
input_file_name = "MATH 308 Project Task One.txt"
input_file = open(input_file_name, 'r')

for line in input_file:
    print("bam!")

input_file.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 2177: invalid start byte

In [13]:
(-1)%5

4

In [23]:
%%timeit

def seed(initial):
    global x_i
    x_i = initial

def lcg():
    a = 7**5
    c = 0
    m = 2**31-1
    global x_i
    x_i = (a*x_i + c) % m
    return x_i

seed(68854*30)

rndm_list=[]
for i in range(50000):
    rndm_list.append(int(lcg()))

20.9 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit

def seed(initial):
    global x_i
    x_i = initial

def lcg():
    a = 7**5
    c = 0
    m = 2**31-1
    global x_i
    x_i = (a*x_i + c) % m
    return x_i

seed(68854*30)

rndm_list = [lcg() for i in range(50000)]

16.6 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
